In [1]:
from tqdm.notebook import tqdm

import json
from Portable import osp

In [2]:
entities_base = r"./entity/merged_entities.json"
depth_base = r"./depth/entities_with_depth.json"
ref_ext_dir = r"./ref_ext"

In [3]:
with open(entities_base, "r", encoding="utf8") as ent_reader,\
    open(depth_base, "r", encoding="utf8") as dep_reader:
    ent_data = json.load(ent_reader)
    dep_data = json.load(dep_reader)

基底字典 REC

In [4]:
base_ent_dict = dict()
for ed in ent_data: base_ent_dict[(ed["image_name"], ed["question"])] = ed

合并深度 DEP

In [5]:
non_directions = list()
for dd in dep_data: 
    key = (dd["image_name"], dd["question"])
    if key not in base_ent_dict: non_directions.append(key); continue
    base_ent_dict[key]["depth"] = dd["depth"]
print(non_directions)

[]


合并参考 REF

In [6]:
from typing import Optional

import spacy
import re

nlp = spacy.load("zh_core_web_md")
base_thresh = 0.5
ref_pattern = r"^参考：(.+)\n类别：([^\n]+)"
def parse_ref(
        ser:str,
        objects:list
    ) -> Optional[dict]:
    if not objects: return None
    result = re.findall(ref_pattern, ser)
    if not result: return None
    result = result[0]
    if result[0] == "画面视角": dref = result[0]
    elif result[0] in objects: dref = result[0]
    else: 
        ref = nlp(result[0])
        simis = [(o, ref.similarity(nlp(o))) for o in objects]
        simis = sorted(simis, key=lambda t: t[1], reverse=True)
        if simis[0][1] < base_thresh: return None
        dref = simis[0][0]
    return {
        "ref": dref,
        "cls": result[1]
    }

non_directions = list(); nullref_counter = 0; nullrefs = list()
for key in tqdm(base_ent_dict.keys()):
    image_json = osp.j(ref_ext_dir, osp.refmt(key[0], ".json"))
    if not osp.exists(image_json): base_ent_dict[key]["reference"] = None; continue
    question = key[1]
    with open(image_json, "r", encoding="utf8") as reader:
        questions = json.load(reader)
    for i, q in enumerate(questions): 
        if question == q["question"]: break
    else: non_directions.append(key); continue
    objects = list(base_ent_dict[key]["objects"].keys())
    parsed_ref = parse_ref(questions[i]["parsed"], objects)
    if not parsed_ref: nullref_counter += 1; nullrefs.append([key, questions[i]["parsed"], objects])
    base_ent_dict[key]["reference"] = parsed_ref
    # break
    
print(f"nullrefs: {nullref_counter}")
print(len(non_directions))

  0%|          | 0/6547 [00:00<?, ?it/s]

/tmp/ipykernel_3534032/4251247544.py:21: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  simis = [(o, ref.similarity(nlp(o))) for o in objects]


nullrefs: 396
1277


In [7]:
# for nr in nullrefs: 
#     for e in nr: print(e)
#     print("-" * 100)
err_dump = r"./ref_rect.json"
with open(err_dump, "w", encoding="utf8") as writer:
    json.dump(nullrefs, writer, indent=4, ensure_ascii=False)

In [8]:
dump_path = r"./merged_tri_infos.json"
seq_ref = r"../sln/VQA_SA/VQA-SA-question.json"
seq_list = list()
with open(seq_ref, "r", encoding="utf8") as reader:
    seq_data = json.load(reader)
    for sd in seq_data: seq_list.append((sd["image_path"].replace("images\\", ""), sd["question"]))
dump_contents = list()
for sle in seq_list: 
    if sle not in base_ent_dict: continue
    dump_contents.append(base_ent_dict[sle])
with open(dump_path, "w", encoding="utf8") as writer:
    json.dump(dump_contents, writer, indent=4, ensure_ascii=False)